# Clone GitHub repository first:

In [1]:
!git clone https://github.com/nocliper/dlts
%cd dlts

Cloning into 'dlts'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (517/517), done.
remote: Compressing objects: 100% (395/395), done.
remote: Total 517 (delta 198), reused 408 (delta 107), pack-reused 0
Receiving objects: 100% (517/517), 11.38 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (198/198), done.
/content/dlts


In [2]:
import matplotlib.pyplot as plt 
import numpy as np
from IPython.display import set_matplotlib_formats

import sys
sys.path.append('functions/')

%matplotlib inline

set_matplotlib_formats('svg')
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# Run file browser and pick data file:

In [3]:
from filebrowser import FileBrowser
File = FileBrowser()
display(File.widget())

path = File.path

# Run interface functions:

In [4]:
def demo(n_windows, X, Bounds, dt, Plot, Extrema, Material, Doping, Smooth):
    """ dfa"""
    import matplotlib.pyplot as plt 
    from matplotlib import cm
    import numpy as np
    from pandas import DataFrame
    from material import material
    from load_data import load_data
    from time_windows import time_windows
    from dlts_plot import dlts_plot
    from find_min import find_min
    from find_max import find_max
    
    axs = []
    fig = plt.figure()
    fig.set_size_inches(10,7)

    axs.append(plt.subplot(211))
    axs.append(plt.subplot(223))
    axs.append(plt.subplot(224))
    axs[2].set_title("Transient")
    axs[0].set_title("DLTS")
    
    
    
    A_e, A_h = material(Material)
    T, Time, C = load_data(File.path, dt)
    T1, T2, = time_windows(n_windows, X, Time, C, axs[2])
    DLTS = dlts_plot(T, Time, C, T1, T2, n_windows, axs[0], Smooth, Bounds)
    
    
    if Plot:
        if Extrema == 'Max':
            find_max(Bounds[0], Bounds[1], DLTS, T, Time, T1, T2, X, n_windows, Doping, A_e, axs[0], axs[1])

        elif Extrema == 'Min':
            find_min(Bounds[0], Bounds[1], DLTS, T, Time, T1, T2, X, n_windows, Doping, A_h, axs[0], axs[1])
            
    np.savetxt('Signal.csv', DLTS, delimiter=',')
    np.savetxt('Temp.csv', np.array(T).T, delimiter=',')
    
    Time = np.asarray(Time)
    T1   = np.asarray(T1)
    T2   = np.asarray(T2)
    tau  = (Time[T2]-Time[T1])/np.log(X)
    
    np.savetxt('Time.csv', tau[:-1], delimiter=',')


In [5]:
def interface():
    '''Initiates widgets'''
    
    from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox, Label
    import ipywidgets as widgets
 
    Material = widgets.Select(
        options = ['Ga2O3', 'GaN'],
        value   = 'Ga2O3',
        description = 'Material:',
        disabled = False)
    
    Doping = widgets.BoundedFloatText(
        value=2.5E18,
        min=1E8,
        max=1E22,
        step=1,
        description=r'Doping $cm^{-3}$:',
        disabled=False)
    
    dt = widgets.BoundedIntText(
        value=150,
        min=15,
        max=350,
        step=1,
        description='Time step',
        disabled=False)

    
    n_windows = widgets.BoundedIntText(
        value=2,
        min= 2, 
        max= 100, 
        step=1, 
        description='№ of windows')

    X = widgets.IntSlider(
        value= 10,
        min=2, #
        max=20, # 
        step=1, # 
        description=r'$t_2/t_1 = $')

    Bounds = widgets.FloatRangeSlider(
        value=[200, 300],
        min=75,
        max=550,
        step=1,
        description=r'T for peaks:',)

    Plot = widgets.ToggleButton(
        value = False,
        button_style='info',
        description = 'Plot arrhenius?',
        disabled = False,
        icon='plus')
    
    Extrema = widgets.Select(
        options = ['Max', 'Min'],
        value   = 'Max',
        description = 'Extrema:',
        disabled = False)
    
    Smooth = widgets.BoundedIntText(
        value=1,
        min= 1, 
        max= 23, 
        step=2, 
        description='Smoothing')

    left_box = VBox([Material, Doping])
    center_box = VBox([dt, n_windows, X, Smooth])
    right_box = VBox([Extrema, Bounds, Plot])
    ui = widgets.HBox([left_box, center_box, right_box])
    out = widgets.interactive_output(demo, {'n_windows':n_windows, 'X':X, 'Bounds':Bounds, 'dt':dt, 'Doping':Doping,
                                            'Plot':Plot, 'Extrema':Extrema, 'Material':Material, 'Smooth':Smooth})
    display(ui, out)


# And run interface

In [6]:
interface()

Output()

In [ ]:
Eg = 4.85 #eV
Ts = ['E1',  'E2*',   'E2',    'E3',    'E8'] #
Ns = [7.28E12, 4.61E13, 5.98E13, 3.77E13,    2.45E12] # cm-3
Es = [0.71,   0.76,  0.85,    1.049,    0.344] # Ec-Et, eV

In [ ]:
from band_plot import band_plot
band_plot(Eg, Ts, Ns, Es, size = 12, thick = 0.05, border = 0.085)